In [1]:
from train import train_setup

In [2]:
trainer = train_setup(r'./train_config.yaml')

100%|██████████| 1000/1000 [00:00<00:00, 10851.26it/s]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: equilibriumivan. Use `wandb login --relogin` to force relogin


In [3]:
# set(trainer.train_dataset.data_names).intersection(set(trainer.eval_dataset.data_names))

In [4]:
trainer.train()

  0%|          | 0/2000 [00:00<?, ?it/s]

{'loss': 0.6655, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.0}


  0%|          | 0/150 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
print(trainer.train_dataset.dataset[0]['input'].shape)
trainer.train_dataset.dataset[0]['target'].shape

In [ ]:
print(trainer.eval_dataset[0]['input'].shape)
trainer.eval_dataset[0]['target'].shape

In [ ]:
trainer.train_dataset.dataset.augmentator()